In [1]:
# Ok so first of all the instructions here (https://cryptopals.com/sets/5/challenges/39) leave a lot out
# And the exercise doesn't work if you follow the instructions specifically... they leave out a lot about the totient function
# I followed Wikipedia instead (https://en.m.wikipedia.org/wiki/RSA_(cryptosystem))


from Crypto.Util.number import getPrime
Prime = lambda : getPrime(1024)

In [2]:
# Using implementations from https://github.com/ricpacca/cryptopals/blob/master/S5C39.py

def GCD(a, b):
    """Computes the greatest common divisor between a and b using the Euclidean algorithm."""
    while b != 0:
        a, b = b, a % b

    return a


def LCM(a, b):
    """Computes the lowest common multiple between a and b using the GCD method."""
    return a // GCD(a, b) * b


def ModInv(a, n):
    """Computes the multiplicative inverse of a modulo n using the extended Euclidean algorithm."""
    t, r = 0, n
    new_t, new_r = 1, a

    while new_r != 0:
        quotient = r // new_r
        t, new_t = new_t, t - quotient * new_t
        r, new_r = new_r, r - quotient * new_r

    if r > 1:
        raise Exception("a is not invertible")
    if t < 0:
        t = t + n

    return t

assert ModInv(17, 3120) == 2753

In [3]:
# Let e be 3
e = 3

In [4]:
# The requirement regarding GCD is specified in Wikipedia, NOT the challenge itself

def GeneratePrimes(e = e):
    while True:
        p, q = Prime(), Prime()
        n = p*q
        et = LCM(p-1, q-1) % n

        if 2 < e < et and GCD(e, et) == 1:
            return p, q, et

p, q, et = GeneratePrimes()

In [5]:
d = ModInv(e, et)
print(d)

assert (d*e) % et == 1

3219221005856495333921099762039309480734372935606363146810030965888147328947196291071784343573752458823727729365310603650046007783264616605138783886765595935283130244799412678139136689684681801007907037775200389020153621665390203682972544849825679115851482056163240623209287602818955490919894841793774459637848623022959013429540826697945180681351458557343144412782559065666573415999936312592502199627425580440623408731690049015182794453925482069894558788097173658690266056579289674655883244141394847964364015222191551446796948591471942772262295577208330202029176577700219626490446240500715867480139512751152792209051


In [6]:
class RSAServer(object):
    def __init__(self, e = 3):
        self.e = e
        p, q, et = GeneratePrimes(e)
        self.p = p
        self.q = q
        self.d = ModInv(e, et)
        self.n = p*q
        
    def GetPubkey(self):
        return self.e, self.n
    
    def DecryptBytes(self, c):
        m = pow(c, self.d, self.n)
        return m.to_bytes(byteorder='big', length = 2*1024//8)
    
    def DecryptInt(self, c):
        m = pow(c, self.d, self.n)
        return m

class RSAClient(object):
    def __init__(self, e, n):
        self.e = e
        self.n = n
        
    def Encrypt(self, message):
        m = message if type(message) is int else int.from_bytes(message, byteorder = 'big')
        assert m < self.n
        return pow(m, self.e, self.n)
        
class Oracle(RSAServer):
    cache = set()
    
    def Decrypt(self, c):
        if hash(c) in self.cache:
            return None
        
        self.cache = { hash(c), *self.cache }
        m = self.DecryptInt(c)
        return m
        
        

In [7]:
server = Oracle()
e, n = server.GetPubkey()
client = RSAClient(e = e, n = n)

In [8]:
# Ok say I'm the first person who is supposed to be decrypting this message

message = b"""
But, soft! what light through yonder window breaks? 
It is the east, and Juliet is the sun.  
Arise, fair sun, and kill the envious moon,
Who is already sick and pale with grief,
That thou her maid art far more fair than she.
"""

ciphertext = client.Encrypt(message)
p = server.Decrypt(ciphertext)
assert type(p) is int
decrypted = p.to_bytes(byteorder='big', length = 2*1024//8).replace(b'\0', b'')
del(p) # To prove I'm not reusing this later in the attack
assert decrypted == message
print(decrypted)

b'\nBut, soft! what light through yonder window breaks? \nIt is the east, and Juliet is the sun.  \nArise, fair sun, and kill the envious moon,\nWho is already sick and pale with grief,\nThat thou her maid art far more fair than she.\n'


In [9]:
# Now pretend to be an attacker.  Show that the oracle won't decrypt the same message again
p = server.Decrypt(ciphertext)
assert p is None

In [10]:
# Now do the attack

s = 123
ciphertext2 = ( pow(s, e, n) * ciphertext ) % n
p2 = server.Decrypt(ciphertext2)

p = ( p2 // s ) % n
decrypted = p.to_bytes(byteorder='big', length = 2*1024//8).replace(b'\0', b'')
assert decrypted == message
print(decrypted)

b'\nBut, soft! what light through yonder window breaks? \nIt is the east, and Juliet is the sun.  \nArise, fair sun, and kill the envious moon,\nWho is already sick and pale with grief,\nThat thou her maid art far more fair than she.\n'
